In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from debuglib import t2n, write_pc

In [3]:
import os, sys
from os.path import join as opj
sys.path.append("../")

import argparse
from omegaconf import OmegaConf
from torch_geometric import data
from torch_points3d.datasets.segmentation.s3dis import S3DIS1x1Ins
import k3d

# paths
# tg_dataset_path = "/home/coder/workspace/data2/torch_point3d/s3dis1x1ins"
tg_dataset_path = 'data/'
# data_config_path = '/home/coder/workspace/code/myrepo/torch-points3d/conf/data/segmentation/s3disfused.yaml'
# config_path = '/home/coder/workspace/code/myrepo/torch-points3d/conf/config.yaml'
output_path = 'output/'

# cfg = OmegaConf.load(config_path)
# cfg.data = OmegaConf.load(data_config_path)
# cfg.data.dataroot = tp3d_dataset_path


In [4]:
dataset = S3DIS1x1Ins(tg_dataset_path)

In [5]:
data = dataset[0]

In [6]:
data

Data(area_room=[4096], block=[4096], ins_y=[4096], pos=[4096, 3], x=[4096, 6], y=[4096])

In [7]:
import numpy as np
import k3d

def to_hex_number(rgb:np.ndarray):
    rgb = rgb[:, 0]*256*256 + rgb[:,1]*256 + rgb[:,2] # from RGB to color code
    return rgb

class VisPoints:
    @staticmethod
    def k3d(xyz, colors:np.ndarray=None, max_color:float=1.0, point_size:float=0.0015):
        # error check
        assert type(colors) == np.ndarray or colors is None
        if colors is not None:
            assert len(colors.shape) in [1, 2], 'Expected colors is rgb (N, 3) or color codes (N).'
            assert len(colors) == len(xyz)

        # color setup
        if colors is not None:
            if not max_color == 255:
                colors = colors / max_color * 255
            if len(colors.shape) == 2: # rgb to color code
                colors = to_hex_number(colors)
            colors = colors.astype(np.uint32)
        else:
            colors = []

        # xyz setup
        if not type(xyz) == np.ndarray:
            xyz = np.array(xyz)
        xyz = xyz.astype(np.float32)

        # plot
        plot = k3d.plot()
        points = k3d.points(xyz, colors, point_size=point_size, shader='flat')
        plot += points
        plot.display()

In [8]:
import torch
def t2n(torch_tensor:torch.Tensor) -> np.ndarray:
    """torch.Tensor to numpy.ndarray
    """
    return torch_tensor.detach().cpu().numpy()

def to_hex_number(rgb:np.ndarray):
    rgb = rgb[:, 0]*256*256 + rgb[:,1]*256 + rgb[:,2] # from RGB to color code
    return rgb


In [9]:
t = data.x[:, :3]
VisPoints.k3d(t2n(data.pos), to_hex_number(t2n(t)))


Output()

In [11]:
data = dataset[0]
print(data)
vis_room_idx = data.area_room[0]
points = []
rgb = []
i = 0
for data in dataset:
    if vis_room_idx != data.area_room[0]:
        break
    i += 1
    points.append(data.pos)
    rgb.append(data.x[:, :3])

print(i)
points = torch.cat(points, dim=0)
rgb = torch.cat(rgb, dim=0)
VisPoints.k3d(t2n(points), to_hex_number(t2n(rgb)))




Data(area_room=[4096], block=[4096], ins_y=[4096], pos=[4096, 3], x=[4096, 6], y=[4096])
55


Output()

In [24]:
data = dataset[0]
print(data)
vis_room_idx = data.area_room[0]
points = []
rgb = []
for data in dataset:
    if vis_room_idx != data.area_room[0]:
        break
    points.append(data.x[:, 3:])
    rgb.append(data.x[:, :3])

points = torch.cat(points, dim=0)
rgb = torch.cat(rgb, dim=0)
VisPoints.k3d(t2n(points), to_hex_number(t2n(rgb)))



Data(area_room=[4096], block=[4096], ins_y=[4096], pos=[4096, 3], x=[4096, 6], y=[4096])


Output()